In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests
from linearmodels import PanelOLS, RandomEffects
from linearmodels.panel import compare
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv('stock_daily_sentiment.csv')
df['date'] = pd.to_datetime(df['date'])
df['stock_code'] = df['stock_code'].astype(str).str.zfill(6)


def granger_analysis(data, stock_code, sentiment_vars, return_vars, maxlag=5):
    """
    执行Granger因果检验
    
    参数:
    data: DataFrame, 包含情绪和收益率数据
    stock_code: str, 股票代码（6位）
    sentiment_vars: list, 情绪变量列表
    return_vars: list, 收益率变量列表
    maxlag: int, 最大滞后阶数
    """
    # 筛选特定股票的数据
    stock_data = data[data['stock_code'] == stock_code].sort_values('date')
    
    results = {}
    for sent_var in sentiment_vars:
        for ret_var in return_vars:
            # 准备检验数据
            test_data = pd.DataFrame({
                'sentiment': stock_data[sent_var],
                'return': stock_data[ret_var]
            }).dropna()
            
            if len(test_data) > maxlag:  # 确保数据量足够
                try:
                    # 执行Granger因果检验
                    granger_test = grangercausalitytests(test_data, maxlag=maxlag, verbose=False)
                    
                    # 存储结果
                    test_results = {}
                    for lag in range(1, maxlag + 1):
                        test_results[lag] = {
                            'ssr_ftest_pvalue': granger_test[lag][0]['ssr_chi2test'][1],
                            'ssr_chi2test_pvalue': granger_test[lag][0]['ssr_chi2test'][1]
                        }
                    
                    results[f"{sent_var}->{ret_var}"] = test_results
                except:
                    continue
    
    return results

# 定义要检验的变量
sentiment_variables = ['avg_sentiment', 'sentiment_std', 'positive_ratio', 
                      'negative_ratio', 'avg_intensity']
return_variables = ['forward_ret_1d', 'forward_ret_3d', 'forward_ret_5d']

# 对每个股票进行Granger因果检验
def run_all_stocks_granger(df):
    all_results = {}
    unique_stocks = sorted(df['stock_code'].unique())  # 排序以保证顺序一致
    
    for stock in unique_stocks:
        print(f"Processing stock: {stock}")
        results = granger_analysis(df, stock, sentiment_variables, return_variables)
        all_results[stock] = results
    return all_results

# 执行分析
granger_results = run_all_stocks_granger(df)

# 结果分析和统计
def analyze_results(granger_results, significance_level=0.05):
    significant_counts = {}
    
    for stock, results in granger_results.items():
        for relation, lags in results.items():
            if relation not in significant_counts:
                significant_counts[relation] = 0
            
            # 检查是否在任何滞后阶数下显著
            for lag, values in lags.items():
                if values['ssr_ftest_pvalue'] < significance_level:
                    significant_counts[relation] += 1
                    break
    
    # 计算显著比例
    total_stocks = len(granger_results)
    significance_ratios = {k: v/total_stocks for k, v in significant_counts.items()}
    
    return significance_ratios

# 输出结果统计
significance_ratios = analyze_results(granger_results)
print("\n显著性统计结果：")
for relation, ratio in significance_ratios.items():
    print(f"{relation}: {ratio:.2%} 的股票显示显著的Granger因果关系")

# 按行业统计结果
def analyze_results_by_industry(df, granger_results, significance_level=0.05):
    industry_stats = {}
    
    # 创建股票代码到行业的映射
    stock_industry_map = df[['stock_code', 'industry']].drop_duplicates().set_index('stock_code')['industry']
    
    # 首先确定所有可能的关系类型
    all_relations = set()
    for stock, results in granger_results.items():
        all_relations.update(results.keys())
    
    # 初始化行业统计字典
    for industry in df['industry'].unique():
        industry_stats[industry] = {relation: 0 for relation in all_relations}
    
    # 统计显著结果
    for stock, results in granger_results.items():
        try:
            industry = stock_industry_map[stock]
            
            for relation, lags in results.items():
                # 检查是否在任何滞后阶数下显著
                for lag, values in lags.items():
                    if values['ssr_ftest_pvalue'] < significance_level:
                        industry_stats[industry][relation] += 1
                        break
        except KeyError:
            print(f"警告: 股票 {stock} 未找到对应的行业信息")
            continue
    
    # 计算每个行业的显著比例
    for industry in industry_stats:
        industry_stock_count = len(df[df['industry'] == industry]['stock_code'].unique())
        if industry_stock_count > 0:  # 防止除以零
            industry_stats[industry] = {k: v/industry_stock_count for k, v in industry_stats[industry].items()}
    
    return industry_stats
# 输出结果统计
significance_ratios = analyze_results(granger_results)
print("\n整体显著性统计结果：")
for relation, ratio in significance_ratios.items():
    print(f"{relation}: {ratio:.2%} 的股票显示显著的Granger因果关系")
# 输出行业统计结果
industry_stats = analyze_results_by_industry(df, granger_results)
print("\n行业显著性统计结果：")
for industry, relations in industry_stats.items():
    print(f"\n行业: {industry}")
    for relation, ratio in relations.items():
        print(f"{relation}: {ratio:.2%} 的股票显示显著的Granger因果关系")


Processing stock: 000002
Processing stock: 000069
Processing stock: 000538
Processing stock: 000568
Processing stock: 000630
Processing stock: 000725
Processing stock: 000858
Processing stock: 000878
Processing stock: 000895
Processing stock: 000933
Processing stock: 000960
Processing stock: 000977
Processing stock: 001979
Processing stock: 002049
Processing stock: 002074
Processing stock: 002129
Processing stock: 002138
Processing stock: 002230
Processing stock: 002304
Processing stock: 002340
Processing stock: 002371
Processing stock: 002410
Processing stock: 002415
Processing stock: 002439
Processing stock: 002460
Processing stock: 002463
Processing stock: 002466
Processing stock: 002475
Processing stock: 002507
Processing stock: 002568
Processing stock: 002594
Processing stock: 300014
Processing stock: 300015
Processing stock: 300035
Processing stock: 300059
Processing stock: 300122
Processing stock: 300146
Processing stock: 300223
Processing stock: 300253
Processing stock: 300274


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests
from linearmodels import PanelOLS, RandomEffects
from linearmodels.panel import compare
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv('stock_daily_sentiment.csv')
df['date'] = pd.to_datetime(df['date'])
df['stock_code'] = df['stock_code'].astype(str).str.zfill(6)


def granger_analysis(data, stock_code, sentiment_vars, return_vars, maxlag=5):
    """
    执行Granger因果检验
    
    参数:
    data: DataFrame, 包含情绪和收益率数据
    stock_code: str, 股票代码（6位）
    sentiment_vars: list, 情绪变量列表
    return_vars: list, 收益率变量列表
    maxlag: int, 最大滞后阶数
    """
    # 筛选特定股票的数据
    stock_data = data[data['stock_code'] == stock_code].sort_values('date')
    
    results = {}
    for sent_var in sentiment_vars:
        for ret_var in return_vars:
            # 准备检验数据
            test_data = pd.DataFrame({
                'sentiment': stock_data[sent_var],
                'return': stock_data[ret_var]
            }).dropna()
            
            if len(test_data) > maxlag:  # 确保数据量足够
                try:
                    # 执行Granger因果检验
                    granger_test = grangercausalitytests(test_data, maxlag=maxlag, verbose=False)
                    
                    # 存储结果
                    test_results = {}
                    for lag in range(1, maxlag + 1):
                        # 存储F统计量和p值
                        test_results[lag] = {
                            'ssr_ftest_stat': granger_test[lag][0]['ssr_ftest'][0],
                            'ssr_ftest_pvalue': granger_test[lag][0]['ssr_ftest'][1],
                            'ssr_chi2test_stat': granger_test[lag][0]['ssr_chi2test'][0],
                            'ssr_chi2test_pvalue': granger_test[lag][0]['ssr_chi2test'][1]
                        }
                    
                    results[f"{sent_var}->{ret_var}"] = test_results
                except:
                    continue
    
    return results

# 定义要检验的变量
sentiment_variables = ['avg_sentiment', 'sentiment_std', 'positive_ratio', 
                      'negative_ratio', 'avg_intensity']
return_variables = ['forward_ret_1d', 'forward_ret_3d', 'forward_ret_5d']

# 对每个股票进行Granger因果检验
def run_all_stocks_granger(df):
    all_results = {}
    unique_stocks = sorted(df['stock_code'].unique())  # 排序以保证顺序一致
    
    for stock in unique_stocks:
        print(f"\nProcessing stock: {stock}")
        results = granger_analysis(df, stock, sentiment_variables, return_variables)
        
        # 打印每个股票的Granger检验统计量
        for relation, lags in results.items():
            print(f"  {relation}:")
            for lag, values in lags.items():
                print(f"    Lag {lag}: F-stat = {values['ssr_ftest_stat']:.4f} (p-value = {values['ssr_ftest_pvalue']:.4f}), "
                      f"Chi2-stat = {values['ssr_chi2test_stat']:.4f} (p-value = {values['ssr_chi2test_pvalue']:.4f})")
        
        all_results[stock] = results
    return all_results

# 执行分析
granger_results = run_all_stocks_granger(df)

# 结果分析和统计
def analyze_results(granger_results, significance_level=0.05):
    significant_counts = {}
    
    for stock, results in granger_results.items():
        for relation, lags in results.items():
            if relation not in significant_counts:
                significant_counts[relation] = 0
            
            # 检查是否在任何滞后阶数下显著
            for lag, values in lags.items():
                if values['ssr_ftest_pvalue'] < significance_level:
                    significant_counts[relation] += 1
                    break
    
    # 计算显著比例
    total_stocks = len(granger_results)
    significance_ratios = {k: v/total_stocks for k, v in significant_counts.items()}
    
    return significance_ratios

# 输出结果统计
significance_ratios = analyze_results(granger_results)
print("\n显著性统计结果：")
for relation, ratio in significance_ratios.items():
    print(f"{relation}: {ratio:.2%} 的股票显示显著的Granger因果关系")

# 按行业统计结果
def analyze_results_by_industry(df, granger_results, significance_level=0.05):
    industry_stats = {}
    
    # 创建股票代码到行业的映射
    stock_industry_map = df[['stock_code', 'industry']].drop_duplicates().set_index('stock_code')['industry']
    
    # 首先确定所有可能的关系类型
    all_relations = set()
    for stock, results in granger_results.items():
        all_relations.update(results.keys())
    
    # 初始化行业统计字典
    for industry in df['industry'].unique():
        industry_stats[industry] = {relation: 0 for relation in all_relations}
    
    # 统计显著结果
    for stock, results in granger_results.items():
        try:
            industry = stock_industry_map[stock]
            
            for relation, lags in results.items():
                # 检查是否在任何滞后阶数下显著
                for lag, values in lags.items():
                    if values['ssr_ftest_pvalue'] < significance_level:
                        industry_stats[industry][relation] += 1
                        break
        except KeyError:
            print(f"警告: 股票 {stock} 未找到对应的行业信息")
            continue
    
    # 计算每个行业的显著比例
    for industry in industry_stats:
        industry_stock_count = len(df[df['industry'] == industry]['stock_code'].unique())
        if industry_stock_count > 0:  # 防止除以零
            industry_stats[industry] = {k: v/industry_stock_count for k, v in industry_stats[industry].items()}
    
    return industry_stats

# 输出整体统计结果
significance_ratios = analyze_results(granger_results)
print("\n整体显著性统计结果：")
for relation, ratio in significance_ratios.items():
    print(f"{relation}: {ratio:.2%} 的股票显示显著的Granger因果关系")

# 输出行业统计结果
industry_stats = analyze_results_by_industry(df, granger_results)
print("\n行业显著性统计结果：")
for industry, relations in industry_stats.items():
    print(f"\n行业: {industry}")
    for relation, ratio in relations.items():
        print(f"{relation}: {ratio:.2%} 的股票显示显著的Granger因果关系")



Processing stock: 000002
  avg_sentiment->forward_ret_1d:
    Lag 1: F-stat = 3.3027 (p-value = 0.0906), Chi2-stat = 4.0105 (p-value = 0.0452)
    Lag 2: F-stat = 0.1588 (p-value = 0.8551), Chi2-stat = 0.4620 (p-value = 0.7937)
    Lag 3: F-stat = 0.5758 (p-value = 0.6469), Chi2-stat = 3.2389 (p-value = 0.3562)
    Lag 4: F-stat = 1.6802 (p-value = 0.2892), Chi2-stat = 18.8180 (p-value = 0.0009)
    Lag 5: F-stat = 1.5853 (p-value = 0.4302), Chi2-stat = 51.5224 (p-value = 0.0000)
  avg_sentiment->forward_ret_3d:
    Lag 1: F-stat = 2.6558 (p-value = 0.1255), Chi2-stat = 3.2249 (p-value = 0.0725)
    Lag 2: F-stat = 3.5172 (p-value = 0.0659), Chi2-stat = 10.2317 (p-value = 0.0060)
    Lag 3: F-stat = 4.0095 (p-value = 0.0516), Chi2-stat = 22.5536 (p-value = 0.0001)
    Lag 4: F-stat = 3.9185 (p-value = 0.0833), Chi2-stat = 43.8868 (p-value = 0.0000)
    Lag 5: F-stat = 2.9998 (p-value = 0.2687), Chi2-stat = 97.4922 (p-value = 0.0000)
  avg_sentiment->forward_ret_5d:
    Lag 1: F-stat =

  avg_sentiment->forward_ret_1d:
    Lag 1: F-stat = 2.2148 (p-value = 0.1589), Chi2-stat = 2.6895 (p-value = 0.1010)
    Lag 2: F-stat = 0.3888 (p-value = 0.6868), Chi2-stat = 1.1311 (p-value = 0.5680)
    Lag 3: F-stat = 0.2028 (p-value = 0.8916), Chi2-stat = 1.1407 (p-value = 0.7673)
    Lag 4: F-stat = 0.8521 (p-value = 0.5490), Chi2-stat = 9.5433 (p-value = 0.0489)
    Lag 5: F-stat = 15.2406 (p-value = 0.0627), Chi2-stat = 495.3183 (p-value = 0.0000)
  avg_sentiment->forward_ret_3d:
    Lag 1: F-stat = 0.0155 (p-value = 0.9026), Chi2-stat = 0.0188 (p-value = 0.8908)
    Lag 2: F-stat = 0.6404 (p-value = 0.5456), Chi2-stat = 1.8630 (p-value = 0.3940)
    Lag 3: F-stat = 0.6472 (p-value = 0.6063), Chi2-stat = 3.6407 (p-value = 0.3030)
    Lag 4: F-stat = 1.2305 (p-value = 0.4039), Chi2-stat = 13.7810 (p-value = 0.0080)
    Lag 5: F-stat = 1.8640 (p-value = 0.3849), Chi2-stat = 60.5797 (p-value = 0.0000)
  avg_sentiment->forward_ret_5d:
    Lag 1: F-stat = 1.4025 (p-value = 0.2560),

  avg_sentiment->forward_ret_1d:
    Lag 1: F-stat = 1.6517 (p-value = 0.2196), Chi2-stat = 2.0056 (p-value = 0.1567)
    Lag 2: F-stat = 0.8302 (p-value = 0.4615), Chi2-stat = 2.4150 (p-value = 0.2989)
    Lag 3: F-stat = 3.3763 (p-value = 0.0750), Chi2-stat = 18.9915 (p-value = 0.0003)
    Lag 4: F-stat = 0.5868 (p-value = 0.6872), Chi2-stat = 6.5725 (p-value = 0.1603)
    Lag 5: F-stat = 0.2265 (p-value = 0.9214), Chi2-stat = 7.3600 (p-value = 0.1952)
  avg_sentiment->forward_ret_3d:
    Lag 1: F-stat = 0.0088 (p-value = 0.9265), Chi2-stat = 0.0107 (p-value = 0.9176)
    Lag 2: F-stat = 1.5877 (p-value = 0.2479), Chi2-stat = 4.6188 (p-value = 0.0993)
    Lag 3: F-stat = 3.1122 (p-value = 0.0885), Chi2-stat = 17.5059 (p-value = 0.0006)
    Lag 4: F-stat = 0.5761 (p-value = 0.6934), Chi2-stat = 6.4521 (p-value = 0.1678)
    Lag 5: F-stat = 0.5162 (p-value = 0.7617), Chi2-stat = 16.7772 (p-value = 0.0049)
  avg_sentiment->forward_ret_5d:
    Lag 1: F-stat = 3.8098 (p-value = 0.0713), C

  avg_sentiment->forward_ret_1d:
    Lag 1: F-stat = 4.4755 (p-value = 0.0528), Chi2-stat = 5.4346 (p-value = 0.0197)
    Lag 2: F-stat = 4.0220 (p-value = 0.0489), Chi2-stat = 11.7002 (p-value = 0.0029)
    Lag 3: F-stat = 1.3969 (p-value = 0.3127), Chi2-stat = 7.8574 (p-value = 0.0491)
    Lag 4: F-stat = 0.7149 (p-value = 0.6164), Chi2-stat = 8.0070 (p-value = 0.0913)
    Lag 5: F-stat = 0.6914 (p-value = 0.6806), Chi2-stat = 22.4718 (p-value = 0.0004)
  avg_sentiment->forward_ret_3d:
    Lag 1: F-stat = 2.1686 (p-value = 0.1630), Chi2-stat = 2.6333 (p-value = 0.1046)
    Lag 2: F-stat = 9.6976 (p-value = 0.0037), Chi2-stat = 28.2112 (p-value = 0.0000)
    Lag 3: F-stat = 5.2748 (p-value = 0.0267), Chi2-stat = 29.6709 (p-value = 0.0000)
    Lag 4: F-stat = 6.7370 (p-value = 0.0301), Chi2-stat = 75.4542 (p-value = 0.0000)
    Lag 5: F-stat = 1.7906 (p-value = 0.3959), Chi2-stat = 58.1941 (p-value = 0.0000)
  avg_sentiment->forward_ret_5d:
    Lag 1: F-stat = 1.5378 (p-value = 0.2353)

  avg_sentiment->forward_ret_1d:
    Lag 1: F-stat = 0.1890 (p-value = 0.6703), Chi2-stat = 0.2295 (p-value = 0.6319)
    Lag 2: F-stat = 0.6369 (p-value = 0.5474), Chi2-stat = 1.8528 (p-value = 0.3960)
    Lag 3: F-stat = 0.7338 (p-value = 0.5605), Chi2-stat = 4.1274 (p-value = 0.2480)
    Lag 4: F-stat = 0.2992 (p-value = 0.8672), Chi2-stat = 3.3510 (p-value = 0.5009)
    Lag 5: F-stat = 0.7631 (p-value = 0.6513), Chi2-stat = 24.8006 (p-value = 0.0002)
  avg_sentiment->forward_ret_3d:
    Lag 1: F-stat = 0.6481 (p-value = 0.4342), Chi2-stat = 0.7870 (p-value = 0.3750)
    Lag 2: F-stat = 0.8431 (p-value = 0.4564), Chi2-stat = 2.4528 (p-value = 0.2934)
    Lag 3: F-stat = 0.7620 (p-value = 0.5463), Chi2-stat = 4.2864 (p-value = 0.2321)
    Lag 4: F-stat = 0.9352 (p-value = 0.5123), Chi2-stat = 10.4748 (p-value = 0.0331)
    Lag 5: F-stat = 1.5573 (p-value = 0.4353), Chi2-stat = 50.6116 (p-value = 0.0000)
  avg_sentiment->forward_ret_5d:
    Lag 1: F-stat = 3.1556 (p-value = 0.0974), C

...

  avg_sentiment->forward_ret_1d:
    Lag 1: F-stat = 24.8309 (p-value = 0.0002), Chi2-stat = 30.1517 (p-value = 0.0000)
    Lag 2: F-stat = 10.0338 (p-value = 0.0033), Chi2-stat = 29.1891 (p-value = 0.0000)
    Lag 3: F-stat = 5.7369 (p-value = 0.0215), Chi2-stat = 32.2702 (p-value = 0.0000)
    Lag 4: F-stat = 2.9189 (p-value = 0.1354), Chi2-stat = 32.6922 (p-value = 0.0000)
    Lag 5: F-stat = 20.9385 (p-value = 0.0462), Chi2-stat = 680.5014 (p-value = 0.0000)
  avg_sentiment->forward_ret_3d:
    Lag 1: F-stat = 2.6890 (p-value = 0.1233), Chi2-stat = 3.2652 (p-value = 0.0708)
    Lag 2: F-stat = 2.2743 (p-value = 0.1491), Chi2-stat = 6.6161 (p-value = 0.0366)
    Lag 3: F-stat = 4.9229 (p-value = 0.0318), Chi2-stat = 27.6912 (p-value = 0.0000)
    Lag 4: F-stat = 4.2141 (p-value = 0.0733), Chi2-stat = 47.1979 (p-value = 0.0000)
    Lag 5: F-stat = 6.7384 (p-value = 0.1343), Chi2-stat = 218.9977 (p-value = 0.0000)
  avg_sentiment->forward_ret_5d:
    Lag 1: F-stat = 0.4925 (p-value = 

  avg_sentiment->forward_ret_1d:
    Lag 1: F-stat = 0.2325 (p-value = 0.6372), Chi2-stat = 0.2823 (p-value = 0.5952)
    Lag 2: F-stat = 1.6163 (p-value = 0.2424), Chi2-stat = 4.7021 (p-value = 0.0953)
    Lag 3: F-stat = 1.5812 (p-value = 0.2683), Chi2-stat = 8.8941 (p-value = 0.0307)
    Lag 4: F-stat = 2.7585 (p-value = 0.1477), Chi2-stat = 30.8950 (p-value = 0.0000)
    Lag 5: F-stat = 0.9510 (p-value = 0.5843), Chi2-stat = 30.9059 (p-value = 0.0000)
  avg_sentiment->forward_ret_3d:
    Lag 1: F-stat = 6.2751 (p-value = 0.0252), Chi2-stat = 7.6198 (p-value = 0.0058)
    Lag 2: F-stat = 0.7870 (p-value = 0.4792), Chi2-stat = 2.2895 (p-value = 0.3183)
    Lag 3: F-stat = 1.4183 (p-value = 0.3071), Chi2-stat = 7.9781 (p-value = 0.0465)
    Lag 4: F-stat = 4.0077 (p-value = 0.0801), Chi2-stat = 44.8860 (p-value = 0.0000)
    Lag 5: F-stat = 2.1115 (p-value = 0.3519), Chi2-stat = 68.6233 (p-value = 0.0000)
  avg_sentiment->forward_ret_5d:
    Lag 1: F-stat = 0.8110 (p-value = 0.3830), 

  avg_sentiment->forward_ret_1d:
    Lag 1: F-stat = 2.5254 (p-value = 0.1343), Chi2-stat = 3.0665 (p-value = 0.0799)
    Lag 2: F-stat = 4.1922 (p-value = 0.0443), Chi2-stat = 12.1954 (p-value = 0.0022)
    Lag 3: F-stat = 4.3944 (p-value = 0.0418), Chi2-stat = 24.7183 (p-value = 0.0000)
    Lag 4: F-stat = 4.6587 (p-value = 0.0612), Chi2-stat = 52.1775 (p-value = 0.0000)
    Lag 5: F-stat = 5.4714 (p-value = 0.1617), Chi2-stat = 177.8212 (p-value = 0.0000)
  avg_sentiment->forward_ret_3d:
    Lag 1: F-stat = 0.0019 (p-value = 0.9659), Chi2-stat = 0.0023 (p-value = 0.9617)
    Lag 2: F-stat = 2.6135 (p-value = 0.1179), Chi2-stat = 7.6030 (p-value = 0.0223)
    Lag 3: F-stat = 5.2017 (p-value = 0.0277), Chi2-stat = 29.2597 (p-value = 0.0000)
    Lag 4: F-stat = 5.4500 (p-value = 0.0456), Chi2-stat = 61.0405 (p-value = 0.0000)
    Lag 5: F-stat = 5.4422 (p-value = 0.1625), Chi2-stat = 176.8702 (p-value = 0.0000)
  avg_sentiment->forward_ret_5d:
    Lag 1: F-stat = 3.2605 (p-value = 0.09

  avg_sentiment->forward_ret_1d:
    Lag 1: F-stat = 5.2524 (p-value = 0.0379), Chi2-stat = 6.3779 (p-value = 0.0116)
    Lag 2: F-stat = 4.5720 (p-value = 0.0359), Chi2-stat = 13.3004 (p-value = 0.0013)
    Lag 3: F-stat = 2.4993 (p-value = 0.1336), Chi2-stat = 14.0584 (p-value = 0.0028)
    Lag 4: F-stat = 1.3724 (p-value = 0.3621), Chi2-stat = 15.3709 (p-value = 0.0040)
    Lag 5: F-stat = 3.2684 (p-value = 0.2507), Chi2-stat = 106.2235 (p-value = 0.0000)
  avg_sentiment->forward_ret_3d:
    Lag 1: F-stat = 5.0199 (p-value = 0.0418), Chi2-stat = 6.0956 (p-value = 0.0136)
    Lag 2: F-stat = 0.6825 (p-value = 0.5255), Chi2-stat = 1.9856 (p-value = 0.3705)
    Lag 3: F-stat = 0.7400 (p-value = 0.5573), Chi2-stat = 4.1627 (p-value = 0.2444)
    Lag 4: F-stat = 1.0013 (p-value = 0.4851), Chi2-stat = 11.2148 (p-value = 0.0243)
    Lag 5: F-stat = 0.6563 (p-value = 0.6957), Chi2-stat = 21.3285 (p-value = 0.0007)
  avg_sentiment->forward_ret_5d:
    Lag 1: F-stat = 6.8099 (p-value = 0.0206

  avg_sentiment->forward_ret_1d:
    Lag 1: F-stat = 3.0603 (p-value = 0.1021), Chi2-stat = 3.7160 (p-value = 0.0539)
    Lag 2: F-stat = 2.4983 (p-value = 0.1275), Chi2-stat = 7.2677 (p-value = 0.0264)
    Lag 3: F-stat = 2.9127 (p-value = 0.1007), Chi2-stat = 16.3841 (p-value = 0.0009)
    Lag 4: F-stat = 0.3530 (p-value = 0.8326), Chi2-stat = 3.9533 (p-value = 0.4124)
    Lag 5: F-stat = 0.3388 (p-value = 0.8576), Chi2-stat = 11.0117 (p-value = 0.0511)
  avg_sentiment->forward_ret_3d:
    Lag 1: F-stat = 0.0158 (p-value = 0.9016), Chi2-stat = 0.0192 (p-value = 0.8897)
    Lag 2: F-stat = 2.5989 (p-value = 0.1190), Chi2-stat = 7.5604 (p-value = 0.0228)
    Lag 3: F-stat = 5.0497 (p-value = 0.0298), Chi2-stat = 28.4045 (p-value = 0.0000)
    Lag 4: F-stat = 0.4113 (p-value = 0.7950), Chi2-stat = 4.6068 (p-value = 0.3301)
    Lag 5: F-stat = 0.6676 (p-value = 0.6908), Chi2-stat = 21.6972 (p-value = 0.0006)
  avg_sentiment->forward_ret_5d:
    Lag 1: F-stat = 0.1261 (p-value = 0.7278), 